In [ ]:
print('jmd shree ganesha')

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import os

In [ ]:
os.getenv('PINECONE_API_KEY')
dataset = r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\data"

In [ ]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf(dataset)

In [ ]:
def text_splitter(data_pdf):
    splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=10)
    chunks = splitter.split_documents(data_pdf)
    
    return chunks

In [ ]:
text_split_chunks = text_splitter(extracted_data)

In [ ]:
text_split_chunks

In [ ]:
print(len(text_split_chunks))

In [ ]:
def text_to_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = text_to_embeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [ ]:
query_result

In [ ]:
client = PineconeVectorStore(index_name = "chat",embedding=embeddings, pinecone_api_key='c5a1e2ec-7282-49f0-aed3-aa96eabfca32')

In [ ]:
t_chunks = [t.page_content for t in text_split_chunks]
index_name = "chat"
doc_test = PineconeVectorStore.from_texts(t_chunks,
            index_name=index_name,
            embedding=embeddings
        )


In [ ]:
doc_test

In [ ]:
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "How can I find out more about your consultants?"

docs=docsearch.similarity_search(query, k=5)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question. You are a helpful assistant who answers questions related to Nuffield Health Hospital.
If the question is not related to Nuffield Health, respond with "I don't know, It's out of my knowledge base.", 
else if you don't know the answer, just say that to contact the helpline number 0330 173 8200. Not to ask for the phone number.

Context: {context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=ctransformers.CTransformers(model=r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])